# if you like it，please vote me ，Thank you,Good score for you（￣︶￣）↗　

The idea from that notebook

https://www.kaggle.com/anyexiezouqu/tpu-enet-b7-xception-densnet201



### References

* https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
* https://codelabs.developers.google.com/codelabs/keras-flowers-data/#0

V1：deeper efficenet 0.965

V2: delete a dense layer and add lr_callback 0.928

V3：delete BN ， add layer ，add max_lr

V5：5K-fold B7

In [ ]:
!pip install -q efficientnet

In [ ]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
import efficientnet.tfkeras as efn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
SEED = 2020

## TPU Config

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)


# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
EPOCHS = 40
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
FOLDS = 5

## Load label and paths

In [ ]:
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sub = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')

train_paths = train.image_id.apply(format_path).values
test_paths = test.image_id.apply(format_path).values

train_labels = train.loc[:, 'healthy':].values
image_size=512
# train_paths, valid_paths, train_labels, valid_labels = train_test_split(
#     train_paths, train_labels, test_size=0.001, random_state=2020)

## Create Dataset objects

A `tf.data.Dataset` object is needed in order to run the model smoothly on the TPUs. Here, I heavily trim down [my previous kernel](https://www.kaggle.com/xhlulu/flowers-tpu-concise-efficientnet-b7), which was inspired by [Martin's kernel](https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu).

In [ ]:
def decode_image(filename, label=None, image_size=(512, 512)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None, seed=2020):
    image = tf.image.random_flip_left_right(image, seed=seed)
    image = tf.image.random_flip_up_down(image, seed=seed)
    image = tf.image.random_contrast(image,lower=0,upper=0.1,seed=seed)
    
    if label is None:
        return image
    else:
        return image, label

# prepare Data

In [ ]:
def get_training_dataset(train_paths,train_labels):
    dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    dataset = dataset.map(decode_image, num_parallel_calls=AUTO)
    dataset = dataset.cache()
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(512)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(valid_paths,valid_labels):
    dataset = tf.data.Dataset.from_tensor_slices((valid_paths, valid_labels))
    dataset = dataset.map(decode_image, num_parallel_calls=AUTO)
    dataset = dataset.cache()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(test_paths):
    dataset = tf.data.Dataset.from_tensor_slices(test_paths)
    dataset = dataset.map(decode_image, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

## Modelling

### Helper Functions

In [ ]:
LR_START = 0.00001
LR_MAX = 0.0001 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 10
LR_SUSTAIN_EPOCHS = 5
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

### Load Model into TPU

In [ ]:
def get_model():
    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB5(
                input_shape=(image_size, image_size, 3),
                weights='imagenet',
                include_top=False
        ),
            L.GlobalAveragePooling2D(),
            L.Dense(train_labels.shape[1], activation='softmax')
    ])
        
        model.compile(
            optimizer = 'adam',
            loss = 'categorical_crossentropy',
            metrics=['categorical_accuracy']
    )
    return model

# 5k-fold

In [ ]:
    STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE
    histories = []
    #models = []
    probs =[]
    auc_score=[]
    #early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)
    kfold = KFold(FOLDS , shuffle = True, random_state = SEED)
    i=0
    for trn_ind, val_ind in kfold.split(train_paths,train_labels):
        print(); print('#'*25)
        print('### FOLD',i+1)
        print('#'*25)
#         print('Train: %s | test: %s' % (train_labels[train_],train_labels[test_indices]))
#         print('Train: %s | test: %s' % (train_paths[train_indices],train_paths[test_indices]))
        trn_paths = train_paths[trn_ind]
        val_paths=train_paths[val_ind]
        trn_labels = train_labels[trn_ind]
        val_labels=train_labels[val_ind]
        model = get_model()
        history = model.fit(
            get_training_dataset(trn_paths,trn_labels), 
            steps_per_epoch = STEPS_PER_EPOCH,
            epochs = EPOCHS,
            callbacks = [lr_callback],#, early_stopping],
            validation_data = get_validation_dataset(val_paths,val_labels),
            verbose=2
        )
        i+=1
        prob = model.predict(get_test_dataset(test_paths), verbose=1)
        probs.append(prob)
        #models.append(model)
        histories.append(history)

In [ ]:
prob_sum = 0
for prob in probs:
    prob_sum += prob
prob_avg = prob_sum/FOLDS
sub.loc[:, 'healthy':] = prob_avg
sub.to_csv('submission.csv', index=False)
sub.head()    